# General data reader for AQUA 
## Streaming mode

The reader includes the ability to simulate a data streaming, that allows to retrieve chunks of data of specific time length.

In [1]:
from aqua import Reader

Let's load some IFS data. We first instantiate a `Reader` object specifying the type of data which we want to read from the catalogue. 
By default, the streaming mode is deactivated (`streaming = False`) and the `retrieve` method will return all the requested data. If you want to activate the streaming mode you should specify the argument `streaming=True`.

There are also three optional parameters: `startdate`, `enddate` and `aggregation`: 

`startdate` : is a string representing the starting date for streaming the data (e.g. '2020-02-25'). If it is not specified, the function will use the first date in the dataset.

`enddate`: is a string representing the ending date for streaming the data (e.g. '2021-01-01').  If it is not specified, the function will use the last date in the dataset.

`aggregation` (str): the streaming frequency.
 Available frequencies: 
- 1S, 5S etc ('S' stands for 'steps', if you want to keep the original data frequency. On hourly data, '5S' will return chunks of 5 hours each.)
  If no 'aggregation' is specified, '1S' will be the default streaming frequency.

- Panda's style: 1M, 7D etc. (keep in mind that the behavior of resampling in Pandas can be a bit counterintuitive when it comes to months (years) due to the variation in the length of months(years). It's safer to ask for multiple chunks of just hours or days.) 

- 'hourly', 'daily', 'weekly', 'monthly', 'yearly'.

- 'hour', 'day', 'pentad', 'week', 'month', 'year', 'decade'  (and their plural).
                                                                    
Keep in mind that if you want to stream chunks of a certain number of days, months etc. you must use the Panda's notation (e.g if you want to stream chunks of three months you should write `aggregation = '3M'`, `aggregation = '3months'` is not accepted).

Let's say we want to stream the data every three days. We need to call:

In [2]:
reader = Reader(model="IFS", exp= "tco2559-ng5", source="ICMGG_atm2d", streaming=True, aggregation = '3D')

Then the actual data are read with the `retrieve` method. By default, the streaming mode is deactivated (`streaming = False`) and the `retrieve` method will return all the requested data. If you want to activate the streaming mode you should specify the argument `streaming=True`.

There are also three optional parameters: `stream_step`, `stream_unit` and `stream_startdate`: 

`stream_unit` is a string representing the unit of time to stream the data by (e.g. 'hours', 'days', 'months') (default is None)

`stream_step` is an integer representing the number of time steps to stream the data by.  Default = 1

`stream_startdate` is a string representing the starting date for streaming the data (e.g. '2020-02-25'). If it is not specified, the function will use the first date in the dataset.

Let's say we want to stream the data every three days. We need to call:

In [3]:
data = reader.retrieve()
data

2023-11-30 16:29:42 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:29:42 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 6)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 0 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 ... 2020-01-22T23:00:00
Dimensions without coordinates: value
Data variables: (12/82)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

We correctly retrieved data from 2020-01-20 to 2020-01-23, keeping the original hourly temporal resolution. 

If `retrieve` is called a second time, it will return the subsequent chunk of 3 days data. This is because the method keeps track of the state of the streaming process through the use of internal attributes.

In [4]:
data = reader.retrieve()
data

2023-11-30 16:29:51 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:29:51 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 6)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 0 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-23 ... 2020-01-25T23:00:00
Dimensions without coordinates: value
Data variables: (12/82)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

By calling `retrieve` a second time, we correctly retrieved data from 2020-01-23 to 2020-01-26.

If the retrieve method in streaming mode is called multiple times, it will return the data in chunks until all of the data has been streamed. The function will automatically determine the appropriate start and end points for each chunk based on the internal state of the streaming process.

If we want to reset the state of the streaming process we can call the `reset_stream` method. This means that if `retrieve` in streaming mode is called again after calling `reset_stream`, it will start streaming the input data from the beginning.

In [5]:
reader.reset_stream()
data = reader.retrieve()
data

2023-11-30 16:29:59 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:29:59 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 6)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 0 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 ... 2020-01-22T23:00:00
Dimensions without coordinates: value
Data variables: (12/82)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

We got back to the first three days, 2020-01-20 to 2020-01-23.

By default, the data stream will start from the first date of the input file.
If we want it to start from a specific date, we can use the parameter `startdate`.

In [6]:
reader = Reader(model="IFS", exp= "tco2559-ng5", source="ICMGG_atm2d", streaming=True, aggregation = '3D', startdate = '2020-05-01')
data = reader.retrieve()
data

2023-11-30 16:30:09 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:30:09 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 6)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 0 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-05-01 ... 2020-05-03T23:00:00
Dimensions without coordinates: value
Data variables: (12/82)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

Similarly, if we want to end the streaming at a specific date, before the natural end of the dataset, we can use the parameter `enddate`.

In [7]:
reader = Reader(model="IFS", exp= "tco2559-ng5", source="ICMGG_atm2d", streaming=True, aggregation = '3D', startdate = '2020-05-01', enddate = '2020-05-05')
data = reader.retrieve()
data

2023-11-30 16:30:19 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:30:19 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 6)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 0 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-05-01 ... 2020-05-03T23:00:00
Dimensions without coordinates: value
Data variables: (12/82)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

In [8]:
data = reader.retrieve()
data

2023-11-30 16:30:25 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:30:25 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


<xarray.Dataset>
Dimensions:  (time: 48, value: 26306560, level: 6)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 0 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-05-04 ... 2020-05-05T23:00:00
Dimensions without coordinates: value
Data variables: (12/82)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

As expected, the second chunk is shorter than the requested 3 days as we have reached the end date.

As mentioned above, if we don't specify the `aggregation` parameter, the data is streamed keeping the the original time resolution of input data
(e.g. hour by hour for hourly data)
If we want chunks of a certain number of steps, we need to specify it. (e.g. aggregations = '6S')


In [9]:
reader = Reader(model="IFS", exp= "tco2559-ng5", source="ICMGG_atm2d", streaming=True, aggregation = '6S')
data = reader.retrieve()
data

2023-11-30 16:30:33 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:30:33 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


<xarray.Dataset>
Dimensions:  (time: 6, value: 26306560, level: 6)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 0 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 ... 2020-01-20T05:00:00
Dimensions without coordinates: value
Data variables: (12/82)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

As requested, we got a six-hours chunk.

Another possibility to deal with data streaming is to call the `generator` method of the `Streaming` class.
This can be done in the Reader initialization through the argument `stream_generator = True`.

In [10]:
reader = Reader(model="IFS", exp= "tco2559-ng5", source="ICMGG_atm2d", stream_generator = 'True', aggregation = 'monthly')
data_gen = reader.retrieve()
data_gen

2023-11-30 16:30:42 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:30:42 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


<generator object Streaming.generator at 0x7f154931ad40>

`data_gen` is now a generator object that yields the requested one-month-long chunks of data. We can do operations with them by iterating on the generator object:

In [11]:
for data in data_gen:
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")

start_date: 2020-01-20T00:00:00.000000000 stop_date: 2020-01-31T23:00:00.000000000
start_date: 2020-02-01T00:00:00.000000000 stop_date: 2020-02-29T23:00:00.000000000
start_date: 2020-03-01T00:00:00.000000000 stop_date: 2020-03-31T23:00:00.000000000
start_date: 2020-04-01T00:00:00.000000000 stop_date: 2020-04-30T23:00:00.000000000
start_date: 2020-05-01T00:00:00.000000000 stop_date: 2020-05-31T23:00:00.000000000
start_date: 2020-06-01T00:00:00.000000000 stop_date: 2020-06-30T23:00:00.000000000
start_date: 2020-07-01T00:00:00.000000000 stop_date: 2020-07-31T23:00:00.000000000
start_date: 2020-08-01T00:00:00.000000000 stop_date: 2020-08-31T23:00:00.000000000
start_date: 2020-09-01T00:00:00.000000000 stop_date: 2020-09-30T23:00:00.000000000
start_date: 2020-10-01T00:00:00.000000000 stop_date: 2020-10-31T23:00:00.000000000
start_date: 2020-11-01T00:00:00.000000000 stop_date: 2020-11-30T23:00:00.000000000
start_date: 2020-12-01T00:00:00.000000000 stop_date: 2020-12-31T23:00:00.000000000
star

An useful feature provided by the 'stream_generator' option is the possibility to have generators with different `startdate` and `enddate`

In [12]:
reader1 = Reader(model="IFS", exp= "tco2559-ng5", source="ICMGG_atm2d", stream_generator = 'True', startdate = '2020-08-01', aggregation = 'monthly')
data_gen1 = reader1.retrieve()
for data in data_gen1:
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")

2023-11-30 16:30:53 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:30:53 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


start_date: 2020-08-01T00:00:00.000000000 stop_date: 2020-08-31T23:00:00.000000000
start_date: 2020-09-01T00:00:00.000000000 stop_date: 2020-09-30T23:00:00.000000000
start_date: 2020-10-01T00:00:00.000000000 stop_date: 2020-10-31T23:00:00.000000000
start_date: 2020-11-01T00:00:00.000000000 stop_date: 2020-11-30T23:00:00.000000000
start_date: 2020-12-01T00:00:00.000000000 stop_date: 2020-12-31T23:00:00.000000000
start_date: 2021-01-01T00:00:00.000000000 stop_date: 2021-01-01T00:00:00.000000000


In [13]:
reader2 = Reader(model="IFS", exp= "tco2559-ng5", source="ICMGG_atm2d", stream_generator = 'True', startdate = '2020-11-01', aggregation = 'monthly')
data_gen2 = reader2.retrieve()
for data in data_gen2:
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")

2023-11-30 16:31:02 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:31:02 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


start_date: 2020-11-01T00:00:00.000000000 stop_date: 2020-11-30T23:00:00.000000000
start_date: 2020-12-01T00:00:00.000000000 stop_date: 2020-12-31T23:00:00.000000000
start_date: 2021-01-01T00:00:00.000000000 stop_date: 2021-01-01T00:00:00.000000000


Another possibility is to use directly the `stream_data` method of the Reader class on a given xarray.Dataset, for example on a subset of the input data:

In [14]:
reader = Reader(model="IFS", exp= "tco2559-ng5", source="ICMGG_atm2d")
data = reader.retrieve()
new_data = reader.stream(data["tprate"], aggregation='3D')
new_data

2023-11-30 16:31:11 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-11-30 16:31:11 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right


<xarray.DataArray 'tprate' (time: 72, value: 26306560)>
dask.array<getitem, shape=(72, 26306560), dtype=float64, chunksize=(1, 26306560), chunktype=numpy.ndarray>
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 ... 2020-01-22T23:00:00
Dimensions without coordinates: value
Attributes: (12/22)
    paramId:                    260048
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    long_name:                  Total precipitation rate
    factor:                     0.2777777777777778
    offset:                     0
    decumulated:                1
    src_units:                  m
    units_fixed:                1